# IN-STK5000/9000 - Medical Project

## Part 1 - Historical Data

Discovering structure in the data. It is uncertain if the symptoms present are all due to
the same disease, or if they are different conditions with similar symptoms. (a) looking at the
data (including symptoms), estimate whether a single-cause model is more likely than a multiplecause model. You can use anything, ranging from histograms or simple clustering algorithms
to a hierarchical Bayesian model.   

***  



### From data readme  

This is historical data in three tables.

X: observations about each patient
A: treatment
Y: outcome of treatment

The data is organised in the following files:

historical.dat: all the tables in one file (matlab format)
historical_X.dat: the X data
historical_A.dat: the A data
historical_Y.dat: the Y data

Modelling the X data can be done through both unsupervised and supervised models. As some of the genome features might be irrelevant, it is probably a good idea to try and filter them out somehow. In later parts of the project, you will be able to perform experiments to narrow done the important genes. For the latter approach, you can combine the last two columns into a classification label, which should give you a cross-validation score of between 60-70%.



### Assumptions  
every person has some disease

say something about confounder variable

Look at clusters, see how they separate features, symptoms  

Biplot on symptoms, look at eigenvalues for components  

https://stackoverflow.com/questions/39216897/plot-pca-loadings-and-loading-in-biplot-in-sklearn-like-rs-autoplot

np.cumsum(pca_model.explained_variance_ratio_)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA, KernelPCA
# from pca import pca as PCA

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score   
import seaborn as sns

import matplotlib.cm as cm

In [ ]:
# for reproducibility
SEED = 1337
np.random.seed(SEED)

Let's take a look at the dataset

In [ ]:
feature_path = "../../data/medical/historical_X.dat"

X = pd.read_csv(feature_path, delimiter=" ", names=["sex", "smoker"] \
                                           + [f"gen_{i}" for i in range(1, 127)] \
                                           + ["symptom_1","symptom_2"])
X

The task is to investigate whether there is a single disease or multiple diseases, and our best cues lies in the symptoms and their causes. For instance, sex and genetic data is assigned at birth, so we know that if there is a prominent correlation between those attributes and the symptoms, we know, assuming that symptoms are not before genes and sex, that these attributes cause the symptoms. However, we cannot assume such on sex. 

Built into pandas, we can use the corr method to find the correlation matrix for each feature.

In [ ]:
X_corr = X.corr()
X_corr

Let's look at the most correlated features with respect to the symptoms

In [ ]:
cond = (X_corr["symptom_1"] > 0.3)  | (X_corr["symptom_1"] < -0.3) | (X_corr["symptom_2"] > 0.3) | (X_corr["symptom_2"] < -0.3)
most_significant = X_corr.loc[cond][["symptom_1", "symptom_2"]][:-2]
sns.heatmap(most_significant, annot=True)

The first symptom seems to be caused by genetical factors more than anything, both positively and negatively. The second symptom did not give us any highly correlated features, which might indicate

In [ ]:
columns = list(X.columns)
X.corr()[["symptom_1", "symptom_2"]][:2]

In [ ]:
reductor = PCA(n_components=2)
x1, x2 = xT = reductor.fit_transform(X).T
plt.scatter(x1, x2, edgecolor="k", alpha=0.4)

In [ ]:
# credit to https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
def biplot(score, coeff , y):
    '''
    Author: Serafeim Loukas, serafeim.loukas@epfl.ch
    Inputs:
       score: the projected data
       coeff: the eigenvectors (PCs)
       y: the class labels
   '''
    xs = score[:,0] # projection on PC1
    ys = score[:,1] # projection on PC2
    n = coeff.shape[0] # number of variables
    plt.figure(figsize=(10,8), dpi=100)
    classes = np.unique(y)
    colors = ['g','r','y']
    markers=['o','^','x']
    for s,l in enumerate(classes):
        plt.scatter(xs[y==l],ys[y==l], c = colors[s], marker=markers[s]) # color based on group
    for i in range(n):
        #plot as arrows the variable scores (each variable has a score for PC1 and one for PC2)
        plt.arrow(0, 0, coeff[i,0], coeff[i,1], color = 'k', alpha = 0.9,linestyle = '-',linewidth = 1.5, overhang=0.2)
        plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'k', ha = 'center', va = 'center',fontsize=10)

    plt.xlabel("PC{}".format(1), size=14)
    plt.ylabel("PC{}".format(2), size=14)
    limx= int(xs.max()) + 1
    limy= int(ys.max()) + 1
    plt.xlim([-limx,limx])
    plt.ylim([-limy,limy])
    plt.grid()
    plt.tick_params(axis='both', which='both', labelsize=14)
    
biplot(xT.T, np.transpose(reductor.components_[0:2, :]), np.zeros(len(x1)))
reductor.components_

In [ ]:
def kmeans_cluster_data(X, k=2, plot=True, fig=None, ax=None):
    """Takes in a dataset and clusters it for the n dimensions of the dataset.
    If plot is enabled, the dataset is then reduced using pca before being plotted in a 2 dimensional space
    Dimensions are kept for the clusterer
    
    returns labels"""
    clusterer = KMeans(n_clusters=k, random_state=SEED)
    labels = clusterer.fit_predict(X)
    
    if plot:
        if not (fig or ax):
            fig = plt.figure(figsize=(14, 7))
            ax = fig.add_subplot()
            
        elif fig:
            ax = fig.add_subplot()
            
        colors = cm.nipy_spectral(labels.astype(float) / k)
            
        ax.set_title(f"{k}-means clustering")
#         pca = PCA(n_components=2)
        pca = PCA(n_components=2)
        
        xp = pca.fit_transform(X)
        centroids = pca.transform(clusterer.cluster_centers_)
#         for i, c in enumerate(centroids):
#             ax.scatter(*xp[labels==i].T, label=f"label {i}", s=30, c=colors, alpha=0.5)
            
        ax.scatter(*xp.T, s=30, c=colors, alpha=0.5)
            
        ax.scatter(*centroids.T, marker='o', c="white", alpha=1, s=200, edgecolor='k')
        for i, c in enumerate(centroids):
            ax.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')
            
#         ax.legend()
    return labels, clusterer

k4labels, _ = kmeans_cluster_data(X.to_numpy(), k=2)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver="lbfgs", multi_class="auto", max_iter=5000)
lr.fit(X, k4labels)
"linear seperability for cluster:", lr.score(X, k4labels)

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    # clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    # cluster_labels = clusterer.fit_predict(X)
    
    cluster_labels, clusterer = kmeans_cluster_data(X, k=n_clusters, ax=ax2)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")
    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

For the clustering part:
* https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
* https://scikit-learn.org/stable/modules/biclustering.html

In [ ]:
# do biclustering and grid-viz

(b) Try and determine whether some particular factors are
important for disease epidemiology and may require further investigations.
You need to be able to validate your findings either through a holdout-set methodology,
appropriately used statistical tests, or Bayesian model comparison.

In [ ]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
sb.set()

In [ ]:
import numpy as np
import pandas as pd

X = pd.read_csv('../../data/medical/historical_X.dat', header=None, sep=" ")
A = pd.read_csv('../../data/medical/historical_A.dat', header=None, sep=" ")
Y = pd.read_csv('../../data/medical/historical_Y.dat', header=None, sep=" ")
X = X.rename(index=str, columns={0: 'gender', 1: 'smoker'})
X = X.rename(index=str, columns={i: 'gene'+str(i-1) for i in range(2,128)})
X_org = X.copy()
X.insert(0, 'symptoms', X[128] + X[129]*2)
X = X.drop(labels=[128, 129], axis=1)
A = A.rename(index=str, columns={0: 'action'})
Y = Y.rename(index=str, columns={0: 'outcome'})

We are assuming symptoms as our target variable. 

Using L1 regularization on Logistic regression for reducing dimensionality.

In [ ]:
Xs = X.drop(labels=['symptoms'], axis=1)
ys = X['symptoms']
C = [10, 1, .5, .2, .1]
for c in C:
    clf = LogisticRegression(penalty='l1', C=c, solver='liblinear', multi_class='ovr')
    fit = clf.fit(Xs, ys)
    print((fit.coef_[:,:]>np.zeros(fit.coef_.shape[1])).sum(axis=1))

Let's check how the features for C=0.1 (41,16,3,5) performs.

In [ ]:
(fit.coef_[:,:]>np.zeros(fit.coef_.shape[1])).sum(axis=0)

Most features appear significant in one class only, a few in two but none in more than 2. 

Using Random Forest for finding feature importance.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem

In [ ]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(Xs, ys)

In [ ]:
deviations = sem([tree.feature_importances_ for tree in clf.estimators_], axis=0)
sorted_feature_ixs = np.argsort(clf.feature_importances_)[::-1]

Let's visualize the feature importance for all features.

In [ ]:
plt.bar(range(Xs.shape[1]), clf.feature_importances_[sorted_feature_ixs],
       color="r", yerr=deviations[sorted_feature_ixs], ecolor='g')
plt.xlabel('Features', fontsize=10)
plt.ylabel('Feature importance score', fontsize=10)
plt.show()

We can see that only few features are more important. Let's zoom in with only first 10 of the important ones.

In [ ]:
plt.bar(range(10), (clf.feature_importances_[sorted_feature_ixs])[:10],
       color="r", yerr=(deviations[sorted_feature_ixs])[:10], ecolor='g')
plt.xticks(range(10), sorted_feature_ixs[:10])
plt.xlabel('Feature number', fontsize=10)
plt.ylabel('Feature importance score', fontsize=10)
plt.show()

Looking at the above graph, we can select the first four (i.e., 5,3,113,11) features which are clearly more relevant than others. 

In [ ]:
X_imp = X.iloc[:,sorted_feature_ixs[:4]].copy()

Let's do clustering with only these important features.


In [ ]:
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import sokalmichener

In [ ]:
X_new = pd.concat([X_imp, X_org.iloc[:,-2:]], axis=1)

In [ ]:
distances = pairwise_distances(X_new, metric=sokalmichener)

In [ ]:
n_cluster = np.arange(1,9)
scores = np.zeros(len(n_cluster))
clusters = np.zeros([len(X_new), len(n_cluster)], dtype=np.int)
for i, n in enumerate(n_cluster):
    clusterer = AgglomerativeClustering(n_clusters=n, affinity='precomputed', 
                                        linkage="average")
    clusters[:,i] = clusterer.fit_predict(distances)
for i in range(1, len(n_cluster)):
    scores[i] = silhouette_score(distances, clusters[:,i], metric='precomputed')
plt.bar(n_cluster[1:], scores[1:])

### Shap values

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(clf)

In [ ]:
expected_value = explainer.expected_value[0]
shap_values = explainer.shap_values(Xs)[0]

In [ ]:
shap.summary_plot(shap_values, Xs)

* Shap values  

* Meaning of epidemiology? Do we predict disease or symptoms? Using which variables?

Measuring the effect of actions. We also observe the effects of two different therapeutic
interventions, one of which is placebo, and the other is an experimental drug. Try and measure
the effectiveness of the placebo versus the active treatment. Are there perhaps cases where the
active treatment is never effective, or should it always be recommended?
1

* Synthetic control 

## Part 2 - Improved Policy

## Part 3 - Adaptive experiment design